In [44]:
import numpy as np
import pandas as pd
from utils import DATA_PATH, customer_hex_id_to_int
# This file builds on the code in https://github.com/radekosmulski/personalized_fashion_recs/blob/main/03c_Basic_Model_Submission.ipynb

In [2]:
transactions = pd.read_parquet(f'{DATA_PATH}/transactions_train.parquet')
customers = pd.read_parquet(f'{DATA_PATH}/customers.parquet')
articles = pd.read_parquet(f'{DATA_PATH}/articles.parquet')

In [3]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Make new features

## Member status (combine `Active` and `FN`)

In [5]:
"""
A   FN      Result
-1  -1      -3
-1   1       1
 1  -1      -1    
 1   1       3
------------------
Result      Meaning             Mapping
-3          NotActive_NoFN      0
 1          NotActive_FN        1
-1          Active_NoFN         2
 3          Active_FN           3
"""
customers['member_status'] = (customers.Active + customers.FN * 2)\
    .replace({-3: 0, -1: 2, 1: 1, 3: 3})

## Time-weighted article popularity

In [8]:
last_date = transactions.t_dat.max()

# Engineered feature 1: item popularity, weighted by amount of weeks since last purchase
transactions['weeks_ago'] = (last_date - transactions.t_dat).dt.days // 7 + 1

# calculate the popularity of each article_id: sum of the number of times it was purchased each week, weighted by weeks_ago
popularity = transactions\
    .groupby(['article_id', 'weeks_ago']).size()\
    .reset_index(name='purchase_count')\
    .groupby('article_id')\
    .apply(lambda x: np.sum(x.purchase_count / x.weeks_ago))\
    .reset_index(name='article_popularity')

transactions = transactions.merge(popularity, how='inner', on='article_id')
transactions.drop(columns=['weeks_ago'], inplace=True)

# Generating candidates

### Last purchase candidates

In [10]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 8.29 s, sys: 109 ms, total: 8.4 s
Wall time: 8.4 s


In [11]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 246 ms, sys: 21.9 ms, total: 268 ms
Wall time: 268 ms


In [12]:
candidates_last_purchase = transactions.copy()

In [13]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 8.78 s, sys: 95.3 ms, total: 8.87 s
Wall time: 8.96 s


### Bestsellers candidates

In [14]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [15]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [16]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [17]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [18]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [19]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [20]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [21]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [22]:
transactions['purchased'] = 1

In [23]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [24]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [25]:
data.purchased.mean()

0.1360000183969228

### Add bestseller information

In [26]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [27]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [28]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [29]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [30]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [31]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [34]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'member_status', 'article_popularity',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [35]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 116 ms, sys: 448 ms, total: 564 ms
Wall time: 809 ms


# Model training

In [36]:
from lightgbm.sklearn import LGBMRanker

In [37]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [38]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.850065
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.183398
[LightGBM] [Debug] init for col-wise cost 0.052230 seconds, init for row-wise cost 0.207357 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1397
[LightGBM] [Info] Number of data points in the train set: 11383971, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
CPU times: user 7.65 s, sys: 2.18 s, total: 9.83 s
Wall time: 5.18 s


In [39]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9816601479090191
article_popularity 0.012186790204185673
article_id 0.003454411979686151
product_type_no 0.000702214256630729
colour_group_code 0.0006936667475777567
section_no 0.0003966379234883029
perceived_colour_master_id 0.0003263479064362502
garment_group_no 0.00019612022128237137
perceived_colour_value_id 0.00019554148792473323
graphical_appearance_no 0.00018812136376886016
fashion_news_frequency 0.0
club_member_status 0.0
age 0.0
member_status 0.0
postal_code 0.0
index_code 0.0
department_no 0.0
index_group_no 0.0


# Calculate predictions

In [40]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs


# Create submission

In [42]:
sub = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

In [45]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 2.14 s, sys: 107 ms, total: 2.25 s
Wall time: 2.27 s


In [46]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [48]:
sub_name = 'submission_lgbm_1'
sub.to_csv(f'{DATA_PATH}/subs/{sub_name}.csv.gz', index=False)

# Evaluation

In [49]:
from evaluation import evaluate
evaluate(f'{DATA_PATH}/subs/{sub_name}.csv.gz')

0.1603984739638478